# 简单数据统计及可视化流程
## 1 统计起诉离婚人群的结婚时长（婚龄）分布
**整体流程：**
1. 从裁判文书中抽取CUS_JHSJ（结婚时间）字段，单位为年，作为原告和被告的结婚时间。然后以JAND（结案年度）作为原告和被告的离婚时间，取差值作为婚龄的数值。
2. 使用matplotlib绘制柱状图观察数据的分布

### 1.1 第一个阶段的任务为观察数据，思考婚龄都需要什么数据，应该如何从原始数据中获得这些数据。

### 1.2 第二个阶段为在一篇样例文档中抽取出原告和被告的结婚时间和结案年度（我们默认结案年度为他们的离婚时间）.

In [ ]:
#样例：从单个文档中抽取结婚时间和结案年度
import csv
import os
import xml.dom.minidom

dir_name = './data'  #数据存放的位置
list_of_file = os.listdir(dir_name)  #得到文件名称列表,在对多个文档进行处理时使用
output = open('marriage_duration.csv','w',encoding = 'utf-8',newline='')  #指定输出目录
fieldnames = ['ID','BEGIN','END','DURATION']
csv_write = csv.DictWriter(output,fieldnames = fieldnames)

path=os.path.join(dir_name,'1640.xml')
if os.path.isfile(path):
    dom = xml.dom.minidom.parse(path)
    root = dom.documentElement
    #结婚时间的抽取
    JHSJ = int(root.getElementsByTagName('CUS_JHSJ')[0].getAttribute("value").split('年')[0])
    #离婚时间的抽取
    LHSJ = int(root.getElementsByTagName('JAND')[0].getAttribute("value"))
    print(LHSJ-JHSJ)
    csv_write.writerow({'ID':path.split('\\')[-1],'BEGIN':JHSJ,'END':LHSJ,'DURATION':LHSJ-JHSJ})
    
output.close() #关闭文件夹

### 1.3 从全部的数据集中抽取符合条件的案件的结婚时间和结案年度，并将得到的结果写入csv表格中，通过查看生成的csv表格检查抽取的数据是否正确。

In [ ]:
#实际使用中对所有文件的处理
for item in list1:
    path = os.path.join(dir_name,item)
    if os.path.isfile(path):
        try:
            dom = xml.dom.minidom.parse(path)
            root = dom.documentElement
            JHSJ = int(root.getElementsByTagName('CUS_JHSJ')[0].getAttribute("value").split('年')[0])
            LHSJ = int(root.getElementsByTagName('JAND')[0].getAttribute("value"))
            DURATION = LHSJ-JHSJ
            if(DURATION in range(0,100)):
                csv_write.writerow({'ID': path.split('\\')[-1], 'BEGIN': JHSJ, 'END': LHSJ,'DURATION':LHSJ-JHSJ})
        except BaseException:
            continue
            #print(item,'未知错误')

### 1.4 使用matplotlib将展现婚龄的分布

In [ ]:
#matplotlib的使用流程
from matplotlib import pyplot

listTest = [12,15,8,9,16,24]

pyplot.bar(range(6),listTest,align = 'center')
 #设置横坐标的文字说明
pyplot.xlabel('x')
#设置纵坐标的文字说明
pyplot.ylabel('y')
#设置标题
pyplot.title('test plot')
pyplot.show()


In [ ]:
#整体婚龄绘图
from matplotlib import pyplot
marriage_duration =[0 for i in range(70)]
inputf2 = open('marriage_duration.csv','r',encoding = 'utf-8')
count = 0
test = []
for line in inputf2:
    duration = line.split(',')[3]
    test.append(int(duration))
for i in range(len(test)):
    n = test[i]
    marriage_duration[n]+=1
    count+=1

pyplot.bar(range(70),marriage_duration,align = 'center')
 #设置横坐标的文字说明
pyplot.xlabel('marriage_duration')
#设置纵坐标的文字说明
pyplot.ylabel('Frequency')
#设置标题
pyplot.title('Frequency chart of marriage_duration')
pyplot.show()

## 2 统计原告与被告有无子女，子女年龄分布
**整体流程：**
1. 首先从裁判文书中抽取CUS_YWZN（有无子女），统计有无子女的案件数量。
2. 如果该字段为有，那么再在CMSSD（查明事实段）中按照正则表达式匹配的方式找到子女的年龄描述，如果查明事实段中描述孩子的语句周围出现了年龄，如“...岁”或者“...周岁”，那么直接使用数值作为孩子的年龄值，如果没有则进一步寻找提到子女的语句周围有没有出现时间，如果有则结合案件的JAND（结案年度）标签计算孩子的年龄，作为最终的年龄值。
3. 绘制孩子的年龄分布图。

### 2.1 第一个阶段的任务为观察数据，有哪些标签字段和子女有关？子女的年龄可以通过哪几种方式得到？

### 2.2 第二个阶段的任务为筛选出有孩子的案件，这里我们规定“CUS_YWZN”字段为判断案件有无子女的标准，即通过判断CUS_YWZN字段是不是“有”来确定案件中是否涉及到子女问题，处理方法同2.2。

### 2.3 通过观察数据尝试从给定的一段文本中抽取出子女的年龄。
涉及内容：正则表达式抽取文本中数据
具体流程:对于CUS_YWZN字段为“有”的裁判文书，抽取CMSSD和JAND备用，在CMSSD中查找包含子，女，孩子的语句，并在包含这些语句的上下文中查找和年龄相关的文本。我们使用两种规则处理这些语句，第一种是直接提到子女年龄的，我们根据周岁和岁进行抽取，第二种是提到子女的出生日期，就需要结合JAND计算子女年龄，当然对文本抽取也可以使用其他的方法。（我们只保留年龄在0-80之间的数据）

In [ ]:
#正则表达式的使用参考https://www.runoob.com/regexp/regexp-syntax.html

In [ ]:
#样例：对于给定文本和JAND的年龄抽取
import re
def get_years(CMSSD,JAND):
    sentences = re.split(r'[，。！？：；、]',CMSSD)   #分句
    years_sentence = []
    for i in range(len(sentences)):
        if('女' in sentences[i] or '子' in sentences[i] or '孩子' in sentences[i]):
            years_sentence.append(sentences[i])
            years_sentence.append(sentences[i-1])
            years_sentence.append(sentences[i+1])
            years_sentence.append(sentences[i-2])
            years_sentence.append(sentences[i+2])
    for sentence in years_sentence:
        if('岁' in sentence or '周岁' in sentence):
            years = re.search(r'(\d+)岁',sentence)
            if not years:
                years = re.search(r'(\d+)周岁',sentence)
            if years:
                if(years.group(1)>0 and years.group(1)<80):
                     return years.group(1)
                break
        elif('生育'in sentence or '生' in sentence):
            years = JAND-int(re.search(r'(\d+)年',sentence).group(1))
            if(years<80 and years>0):
                return years
            break
    return None

CMSSD = "原告王某甲诉称，原被告经人2004年2月15日生儿子王某丁。原被告未经充分了解即草率结婚，婚后感情一般，经常争吵，被告没有家庭责任感，经常殴打原告，导致夫妻感情彻底破裂。请判令原被告离婚；儿子王某丁随原告生活，被告承担抚养费；依法分割夫妻共同财产。 被告王某乙辩称，我不同意离婚。我对家庭是负责任的。原告说的不是事实，过去我包地种地，现在打工，挣的钱都给她，我没有打过她。 经审理查明，原被告经人介绍于1998年2月份相识，于××××年××月××日登记结婚，于×2004年2月15日生儿子王某丁。原被告均系再婚，婚后感情一般，时常为琐事吵嘴打架。2015年6月8日双方为琐事发生争吵，原告回娘家居住，分居至今。现原告起诉离婚，被告不同意离婚，经本院做和好工作未果。 上述事实有原被告结婚证、庭审笔录证实。"
print(get_years(CMSSD,2015))


### 2.4  从全部的数据集中抽取符合条件子女年龄，并将得到的结果写入csv表格中，通过查看生成的csv表格检查抽取的数据是否正确。

In [ ]:
#有无子女及子女大小
import os,shutil
import  xml.dom.minidom
import csv
import re

dir_name = './data'
list1 = os.listdir(dir_name)
output = open('child_years.csv','w',encoding = 'utf-8',newline='')
fieldnames = ['ID','CHILD','YEARS',]
csv_write = csv.DictWriter(output,fieldnames = fieldnames)
countYes = 0
countNo = 0
for item in list1:
    path=os.path.join(dir_name,item)
    if os.path.isfile(path):
        try:
            dom = xml.dom.minidom.parse(path)
            root = dom.documentElement
            YWZN = root.getElementsByTagName('CUS_YWZN')[0].getAttribute("value")
            JAND = int(root.getElementsByTagName('JAND')[0].getAttribute("value"))
            YEARS = 0
            if(YWZN == '有'):
                CMSSD = root.getElementsByTagName('CMSSD')[0].getAttribute("value")
                YEARS = get_years(CMSSD,JAND)
                if(YEARS):
                    countYes+=1
            
            else:
                Years = None
                countNo +=1
                
            csv_write.writerow({'ID':path.split('\\')[-1],'CHILD':YWZN,'YEARS':YEARS})
        except BaseException:
            print(item,'未知错误')
print(countYes,'有孩子的案件数量')
print(countNo,'没有孩子的案件数量')       
output.close()

def get_years(CMSSD,JAND):
    sentences = re.split(r'[，。！？：；、]',CMSSD)   #分句
    years_sentence = []
    for i in range(len(sentences)):
        if('女' in sentences[i] or '子' in sentences[i] or '孩子' in sentences[i]):
            years_sentence.append(sentences[i])
            years_sentence.append(sentences[i-1])
            years_sentence.append(sentences[i+1])
            years_sentence.append(sentences[i-2])
            years_sentence.append(sentences[i+2])
    for sentence in years_sentence:
        if('岁' in sentence or '周岁' in sentence):
            years = re.search(r'(\d+)岁',sentence)
            if not years:
                years = re.search(r'(\d+)周岁',sentence)
            if years:
                if(years.group(1)>0 and years.group(1)<80):
                     return years.group(1)
                break
        elif('生育'in sentence or '生' in sentence):
            years = JAND-int(re.search(r'(\d+)年',sentence).group(1))
            if(years<80 and years>0):
                return years
            break
    return None







### 2.5 使用matplotlib将展现子女年龄的分布
matplotlib的使用同1.4中的例子

In [ ]:
#子女年龄分布
from matplotlib import pyplot
prison_list =[0 for i in range(61)]
# for i in range(1,301):
#     prison_list.append(i)
inputf2 = open('child_years.csv','r',encoding = 'utf-8')
count = 0
test = []
for line in inputf2:
    childYears = line.split(',')[2]
    if childYears == '\n'or int(childYears)==0:
        continue
    test.append(int(childYears))
#print(max(test),min(test))
#print(test)
for i in range(len(test)):
    n = test[i]
    prison_list[n]+=1
    count+=1
print(prison_list)
print(count)

pyplot.bar(range(61),prison_list,align = 'center')
 #设置横坐标的文字说明
pyplot.xlabel('child_years')
#设置纵坐标的文字说明
pyplot.ylabel('Frequency')
#设置标题
pyplot.title('Frequency chart of childYears')
pyplot.show()